In [46]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
cd /content/drive/Othercomputers/laptop/jupyter/

/content/drive/Othercomputers/laptop/jupyter


In [48]:
import random
import data
import os
import parameters as p
import tensorflow as tf
import importlib
import viewer as v
import numpy as np
import pickle

In [49]:
importlib.reload(p)
importlib.reload(data)

<module 'data' from '/content/drive/Othercomputers/laptop/jupyter/data.py'>

In [50]:
random.seed()
classes = data.retrieve_class_names()
output_dim = p.GX * p.GY * (5 + len(classes))
# pages = data.Retriever().retrieve(classes)

In [51]:
def generate_data(data_dir_path, batch_size, output_size):
    files = os.listdir(data_dir_path)
    pad = []
    for file in files:
        with open(data_dir_path + b'/' + file, 'rb') as data_file:
            part_data = pickle.load(data_file)
        part_data = pad + part_data
        pad_size = len(part_data) % batch_size
        batches_number = int(len(part_data) / batch_size)
        pad = part_data[len(part_data) - pad_size:]
        part_data = part_data[:len(part_data) - pad_size]
        for j in range(batches_number):
            batch = part_data[j * batch_size:(j + 1) * batch_size]
            yield (np.reshape(np.array([i for i, j in batch]), (batch_size, p.X, p.Y, 3)),
                   np.reshape(np.array([j for i, j in batch]), (batch_size, output_size)))

In [52]:
files = os.listdir(p.PNG_PATH)
# generator = data.Generator(pages, output_dim, classes)

ds = tf.data.Dataset.from_generator(
    generate_data,
    args=['/content/drive/MyDrive/noteheadBlack', p.BATCH_SIZE, output_dim],
    output_types=(tf.int32, tf.float32),
    output_shapes=([p.BATCH_SIZE, p.Y, p.X, 3], [p.BATCH_SIZE, output_dim]))

In [53]:
from keras import backend as K

def get_loss(y_actual,y_pred):

    y_actual_reshaped = K.reshape(y_actual, [p.BATCH_SIZE * p.GX * p.GY, -1])
    y_pred_reshaped = K.reshape(y_pred, [p.BATCH_SIZE * p.GX * p.GY, -1])

    main_probs_act = y_actual_reshaped[..., 0:1]
    main_probs_pred = y_pred_reshaped[..., 0:1]
    xy_probs_act = y_actual_reshaped[..., 1:3]
    xy_probs_pred = y_pred_reshaped[..., 1:3 ]
    wh_probs_act = y_actual_reshaped[..., 3:5]
    wh_probs_pred = y_pred_reshaped[..., 3:5 ]
    classes_probs_act = y_actual_reshaped[..., 5:]
    classes_probs_pred =  y_pred_reshaped[..., 5:]

    loss = 4 * K.sum(K.abs(main_probs_act - main_probs_pred))
    loss += 4 * K.sum(K.sum(K.abs(xy_probs_act - xy_probs_pred) * main_probs_act, 1) )
    loss += K.sum(K.sum(K.abs(wh_probs_act - wh_probs_pred) * main_probs_act, 1) )
    # loss += K.sum(K.sum(K.abs(classes_probs_act - classes_probs_pred), 1))

    return loss

#%
# import numpy as np 
#
# y_actual = np.array([[1,0,0.1,0.1,0.1,1],[0,1,0,0,0,0]])
# y_pred = np.array([[1,0.1,0.1,0.1,0.1,1],[0,1,1,1,1,0]])
# loss = get_loss(y_actual, y_pred)

In [54]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(p.Y,p.X, 3))

model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False, weights='imagenet',
    input_tensor=inputs
)

x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(output_dim, activation="relu", name="pred")(x)
model = tf.keras.Model(inputs, outputs)


In [ ]:
importlib.reload(p)
importlib.reload(data)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.9)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=get_loss,
              metrics=tf.keras.metrics.MeanAbsoluteError(name="mean_absolute_error", dtype=None))
a = model. fit(ds, epochs=10)

Epoch 1/10
10240/10240 [==============================] - 787s 76ms/step - loss: 214.1894 - mean_absolute_error: 0.2537
Epoch 2/10
10240/10240 [==============================] - 759s 74ms/step - loss: 198.1307 - mean_absolute_error: 0.2422
Epoch 3/10
10240/10240 [==============================] - 765s 75ms/step - loss: 196.1903 - mean_absolute_error: 0.2410
Epoch 4/10
10240/10240 [==============================] - 758s 74ms/step - loss: 195.6611 - mean_absolute_error: 0.2408
Epoch 5/10
 4320/10240 [===========>..................] - ETA: 7:19 - loss: 192.6303 - mean_absolute_error: 0.2404

In [ ]:
model.save_weights("aaa/a1")

In [ ]:
g = generate_data(b'/content/drive/MyDrive/noteheadBlack', p.BATCH_SIZE, output_dim)

In [ ]:
aa = g.__next__()
aaa = model.predict(aa[0])
importlib.reload(v)
v.show_prediction(aaa, aa[0])